Creator: Yen Ting Lin, CCS-3, LANL 

Note: For the manuscript "*Gene expression noise accelerates the evolution of a biological oscillator*", co-authored by Nicolas E. Buchler, NCSU

The code has been reviewed by Richard P. Feynman Center for Innovation at the Los Alamos National Laboratory, with a C number C21109

This notebook simulates the evolutionary processes of a single biophysical parameter $\beta^F_X$ for stochastic gene expression dynamics.

Prerequesite: (1) compile ```/CTMC_simulation/titrationOscillator.cpp``` to an executable t_evolution.out in the current folder, and (2) create a folder named ```evoBuffer``` for saving the generated sample paths.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from matplotlib import cm 

from multiprocessing import Pool
from subprocess import call, STDOUT

from scipy import signal
from scipy.stats import norm, uniform

In [ ]:
plt.rcParams.update({'font.size':14})

In [ ]:
populationN = 100
poolN = populationN

In [ ]:
def run_model(par):
    
    lf = open('./evoBuffer/'+str(par[0])+'.log','w')
    parameters=f'{par[1]} 200 1000 50 0.005 {par[0]}'   # par[1]: betaFX, par[0]: unique ID
    call("./t_evolution.out %s"%parameters,shell=True,stdout=lf,stderr=STDOUT)
    lf.close()
    

In [ ]:
def compileParList(par):
    
    populationN = len(par)
    
    parList = np.zeros((populationN, 2))
    
    parList[:,0] = range(populationN)
    parList[:,1] = par[:]
    
    return parList

In [ ]:
def batchRun(parList):
    
    pool = Pool(processes=poolN)
    pool.map(run_model,parList)
    pool.close()

In [ ]:
def evaluatePeak(populationN):
    
    output = []
    
    output_fullPSD = []
    
    for i in range(populationN):
    
        data = np.genfromtxt('./evoBuffer/titrationOscillator-'+str(i)+'.txt')
        x=data[5000:,3]

        dt = data[1,0]-data[0,0]
        x=x-np.mean(x)
        fs = 1/dt
        freqTemp, psdTemp = signal.welch(x, fs, nperseg=256, nfft=len(x), window='boxcar', noverlap=0)

        maxDensity = np.amax(psdTemp)
        index = np.where(psdTemp==maxDensity)[0][0]
        maxFrequency = freqTemp[index]

        if maxDensity < 1E-4:
            
            maxDensity = 0    
            maxFrequency = 0
            
            
        output.append([maxFrequency, maxDensity])
        
        output_fullPSD.append(np.vstack((freqTemp, psdTemp)))

    return np.array(output),  np.array(output_fullPSD)

In [ ]:
def evaluateFitness(parList):
    
    batchRun(parList)
    
    populationN = len(parList)
    l,_ = evaluatePeak(populationN)
    
    # combine peak frequency and peak power
    targetF = 1.5

    C0 = 0
    C1 = 1
    
    #return -C1*(l[:,0]-targetF)**2 + l[:,1]
    return l[:,1]/(1+l[:,1])/(1+(l[:,0]-targetF)**2)

# Evolutionary process

In [ ]:
def selectionMutation(parList,mutationSTD):
    
    populationN = len(parList)
    
    selectionPercentage = 10.
    
    selectedN = np.around(populationN*(selectionPercentage/100)).astype('int64')
    
    # Evaluate the fitness
    f = evaluateFitness(parList)

    # Order and select
    selectedIndex = f.argsort()[::-1][:selectedN]

    # select
    selectedPar = parList[selectedIndex,1]
    
    # mutation kernel
    
    newParList = np.zeros((populationN, 2))
    newParList[:,0] = range(populationN)
    
    for i in range(populationN):
        
        # randomly select one parent
        
        parParent = selectedPar[np.random.choice(selectedN)]
        newParList[i,1] = np.random.normal(loc = parParent, scale = mutationSTD)

    return newParList,f
    
    

In [ ]:
# generate N paths
pathN = 1
tN = 10

summary = np.zeros([2, pathN, tN, populationN])

for pp in range(pathN):

    initialSTD = 0.2
    mutationSTD = 0.2

    initialPar = norm(loc=25, scale=initialSTD).rvs(populationN)
    
    parList = compileParList(initialPar)

    for tt in range(tN):

        print(f'Path {pp}, generation {tt}')

        parList,fitness = selectionMutation(parList,mutationSTD=mutationSTD)
        
        summary[0, pp, tt, :] = parList[:,1]
        summary[1, pp, tt, :] = fitness[:]
        


In [ ]:
np.savez('titrationOscilattor-summary-sto-betaFX', summary=summary)

In [ ]:
for i in range(pathN):
    
    series = []
    
    for t in range(tN):
        
        fits = summary[1,i,t,:]
        pars = summary[0,i,t,:]
    
        series.append(pars[np.where(fits==np.amax(fits))[0][0]])
        
    plt.plot(range(tN), series)
    
plt.xlabel('Generation')
plt.ylabel('$\\beta^F_X$')      